In [5]:
from MKLpy.preprocessing import normalization, rescale_01
from sklearn.metrics.pairwise import rbf_kernel as RBF
import os
import pandas as pd
import numpy as np
import pickle as pkl
import aknotebooks.classification.convenience_functions.mkl_base as mkl_base
from MKLpy.metrics.pairwise.misc import homogeneous_polynomial_kernel as HPK_kernel
from aknotebooks.classification.convenience_functions.mkl_base import hardDrivesLoc, dataOnlyDrive, folderList, symbols, \
    dataList, finalLocation, DataLoc, path
# from aknotebooks.classification.convenience_functions.fit_mkl import featureCreation
from aknotebooks.classification.convenience_functions.mkl_base import CreateElements
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [6]:

def featureCreation(idxKey, locDict):
    """ gives out clean features and labels for a given locDict and a idxKey """
    featuresIdxDirFileLoc = locDict[idxKey][0]
    labelsIdxDirFileLoc = locDict[idxKey][1]
    ''' read the features file'''
    featuresTupleFile = pkl.load(open(featuresIdxDirFileLoc, "rb"), encoding='latin1')
    dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1], \
                            featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)
    ''' read the labels file'''
    labelsDf = pd.read_csv(labelsIdxDirFileLoc)
    ''' pop the labels out'''
    labels = labelsDf['label_PrMov__window_5__thres_arbitrary__0.1']
    '''dataframe of Features and Labels - X and Y'''
    dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
    labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
    ''' drop the labels from the features'''
    dfX = dfXY.drop(columns=[labelName])
    arrX = np.array(dfX)
    ''' feature normalisation'''
    # feature scaling in [0,1] - X = rescale_01(arrX)
    X = normalization(rescale_01(arrX))
    y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
    ''' returns features, labels'''
    return X, y

In [7]:

MKLExpPath = os.path.join(DataLoc, path)
symbols = sorted(os.listdir(MKLExpPath))


symbolIdx = 3
print(symbols[symbolIdx])
MKLSymbolPath = os.path.join(MKLExpPath, symbols[symbolIdx])
MKLSymbolKernelsPath = "/".join((MKLSymbolPath, 'Kernels'))


AZN.L


In [20]:
MKLSymbolPath

'/media/ak/WorkDrive/Data/MKL_Experiments/AZN.L'

In [8]:
cleanListKernelInputKeys = {}
compKernels={}

''' creating the appropriate paths and making the code far more modular'''
SymbolCommonPaths = mkl_base.open_pickle_file(MKLSymbolPath,'LocDictsListCorrect.pkl')  # where the locations of the symbol
# paths are
kernelInputPaths = mkl_base.open_pickle_file(MKLSymbolPath, 'kernelInputsLocations.pkl')  # where the location of the kernel


In [23]:
kernelFiles =[s for s in os.listdir("/".join((MKLSymbolPath,'Kernels'))) if ('RBFKernels.pkl') in s]


for keyInput in kernelInputPaths.keys() :
    Xtr, ytr = featureCreation(idxKey=keyInput,
                                   locDict=SymbolCommonPaths)
    if (np.isfinite( Xtr ).all()==True and  np.isnan( Xtr ).all() == False):
        print(keyInput)
        cleanListKernelInputKeys[keyInput] = SymbolCommonPaths[keyInput]
    else:
        print('Shapes dont match.')
        continue


pkl.dump(cleanListKernelInputKeys, open("/".join((MKLSymbolKernelsPath, "cleanKernelsList.pkl")), "wb"))

In [13]:
cleanListKernelInputKeys = pkl.load(open("/".join((MKLSymbolKernelsPath, "cleanKernelsList.pkl",)),"rb"), encoding='latin1')


In [16]:
keysKernels = list(cleanListKernelInputKeys)

In [26]:
compKernels={}
RBFKernels={}
for kernelKey in cleanListKernelInputKeys:
    featuresIdxDirFileLoc = cleanListKernelInputKeys[kernelKey][0]
    labelsIdxDirFileLoc = cleanListKernelInputKeys[kernelKey][1]
    
    featuresTupleFile = pkl.load(open(featuresIdxDirFileLoc, "rb"), encoding='latin1')
    dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1], \
                                featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)
    ''' read the labels file'''
#     labelsDf = 
    ''' pop the labels out'''
    labels = pd.read_csv(labelsIdxDirFileLoc)['label_PrMov__window_5__thres_arbitrary__0.1']
    '''dataframe of Features and Labels - X and Y'''
    dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
    labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
    ''' drop the labels from the features'''
    dfX = dfXY.drop(columns=[labelName])
    ''' feature normalisation'''
    # feature scaling in [0,1] - X = rescale_01(arrX)
    Xtr = normalization(rescale_01(np.array(dfX)))
    #kernelHPK = [HPK_kernel(Xtr, degree=d) for d in range(1, 4)]
    kernelRBF = [RBF(Xtr, gamma=gamma) for gamma in [1., 10, 100.]]
    print('training EasyMKL...for polynomials and RBF', end='')
    #compKernelsRBF[kernelKey] = [kernelHPK, kernelRBF]
    RBFKernels[kernelKey] = [kernelRBF]
    
    
    
    
    
# kernelFileName = "/".join(
#         (MKLSymbolKernelsPath, "".join((kernelKey[0], '_', kernelKey[1], "_RBFKernels.pkl"))))
    
kernelFileName = "/".join(
        (MKLSymbolKernelsPath, symbols[symbolIdx], "_RBFKernels.pkl"))
pkl.dump(RBFKernels[kernelKey], open(kernelFileName, "wb"))

           
 
    
    

training EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials and RBFtraining EasyMKL...for polynomials

MemoryError: 

In [10]:
len(cleanListKernelInputKeys)

1705

In [ ]:
for kernelKey in cleanListKernelInputKeys:
    featuresIdxDirFileLoc = cleanListKernelInputKeys[kernelKey][0]
    labelsIdxDirFileLoc = cleanListKernelInputKeys[kernelKey][1]
    
    featuresTupleFile = pkl.load(open(featuresIdxDirFileLoc, "rb"), encoding='latin1')
    dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1], \
                                featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)
    ''' read the labels file'''
#     labelsDf = 
    ''' pop the labels out'''
    labels = pd.read_csv(labelsIdxDirFileLoc)['label_PrMov__window_5__thres_arbitrary__0.1']
    '''dataframe of Features and Labels - X and Y'''
    dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
    labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
    ''' drop the labels from the features'''
    dfX = dfXY.drop(columns=[labelName])
    ''' feature normalisation'''
    # feature scaling in [0,1] - X = rescale_01(arrX)
    Xtr = normalization(rescale_01(np.array(dfX)))
    ytr = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]

In [ ]:
ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]

In [28]:
RBFKeys=list(RBFKernels)
print(RBFKeys[0]) #load this data and this kernel and fit model
fwdKeys = [i for i in RBFKeys if i>RBFKeys[0]] #load this data and produce the model!

('20170829', '20170925')


In [ ]:
if Xtr.shape[0] == ytr.shape[0]:
            print(SymbolCommonPaths[keyInput])
            print('doing the kernels bit')
            kernelHPK = [HPK_kernel(Xtr, degree=d) for d in range(1, 11)]
            kernelRBF = [RBF(Xtr, gamma=gamma) for gamma in [1., 10, 100.]]
            print('training EasyMKL...for polynomials and RBF', end='')
            compKernels[keyInput] = [kernelHPK, kernelRBF]
        else:
            print('Shapes dont match.')
            continue

In [ ]:
kernelFileName = "/".join((MKLSymbolPath,'kernelInputsLocations.pkl'))
pkl.load(open(kernelFileName, "rb"))

In [ ]:
kernelFileName

In [ ]:
fileName = "/".join((MKLSymbolPath,'LocDictsListCorrect.pkl'))
pkl.load(open(fileName, "rb"))

In [29]:
list(RBFKernels)
for kernelKey in RBFKernels:
    featuresIdxDirFileLoc = cleanListKernelInputKeys[kernelKey][0]
    labelsIdxDirFileLoc = cleanListKernelInputKeys[kernelKey][1]
    
    featuresTupleFile = pkl.load(open(featuresIdxDirFileLoc, "rb"), encoding='latin1')
    dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1], \
                                featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)
    ''' read the labels file'''
#     labelsDf = 
    ''' pop the labels out'''
    labels = pd.read_csv(labelsIdxDirFileLoc)['label_PrMov__window_5__thres_arbitrary__0.1']
    '''dataframe of Features and Labels - X and Y'''
    dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
    labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
    ''' drop the labels from the features'''
    dfX = dfXY.drop(columns=[labelName])
    ''' feature normalisation'''
    # feature scaling in [0,1] - X = rescale_01(arrX)
    Xtr = normalization(rescale_01(np.array(dfX)))
    ytr = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
    ker_list = RBFKernels[kernelKey]
    

In [ ]:
    clfEasy = EasyMKL(lam=0.1).fit(KLtr,ytr)#combining kernels with the EasyMKL algorithm
    clfRBF = EasyMKL(lam=0.1).fit(ker_list,ytr)
    print('Average Kernel Testing')
y_pred = clf.predict(KLte)                 #predictions
y_score = clf.decision_function(KLte)      #rank
accuracy = accuracy_score(Yte, y_pred)
fprAverage, tprAverage, thresholdsAverage = roc_curve(Yte.ravel(), y_score.ravel())
roc_auc = auc(fprAverage, tprAverage)

print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))


''' Test Linear'''
print('MKL-Linear Testing')
y_predMKLLinear = clfEasy.predict(KLte)                 #predictions
y_scoreMKLLinear = clfEasy.decision_function(KLte)  #rank

accuracy_MKLLinear = accuracy_score(Yte, y_predMKLLinear)
roc_auc_MKLLinear = roc_auc_score(Yte, y_scoreMKLLinear)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy_MKLLinear, roc_auc_MKLLinear))

fprLinear, tprLinear, thresholds = roc_curve(Yte.ravel(), y_scoreMKLLinear.ravel())

roc_auc_Linear = auc(fprLinear, tprLinear)

# fprMKLLinear, fprMKLLinear, thresholdsRBF =roc_curve(Yte, y_scoreMKLLinear)

print(roc_auc_Linear)